##10.08 - 연관규칙 

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
dataset=[['사과','치즈','생수'],
['생수','호두','치즈','고등어'],
['수박','사과','생수'],
['생수','호두','치즈','옥수수']]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경

In [ ]:
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [ ]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
frequent_itemsets 

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(생수, 호두)"
7,0.50,"(호두, 치즈)"
8,0.50,"(생수, 호두, 치즈)"


In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

NameError: ignored

In [ ]:
course = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.21_course.csv",index_col=0)
course.head(2)

,id,places,theme,cluster,score
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116


In [ ]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_attraction.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...


In [ ]:
course['places'] = course['places'].apply(lambda x: x[1:-1].split(','))

In [ ]:
for val in course['places'].values:
  print(val)
  print(type(val))
  print(val[0])
  print(type(val[0]))
  break

['602', ' 10', ' 58', ' 29']
<class 'list'>
602
<class 'str'>


###id->title로 변환

In [ ]:
course['places'] = course['places'].apply(lambda x : list(map(int,x)))

In [ ]:
def inner_parse(x):
  title = attraction[attraction['id']==x]['title'].values[0]
  return title

print(list(map(inner_parse,[602,  10,  58,  29,])))    

['수목원테마공원(야시장길&LED공원)', '카멜리아힐', '성읍민속마을', '아쿠아플라넷 제주']


In [ ]:
course['place_title'] = course['places'].apply(lambda x : list(map(inner_parse, x)))

In [ ]:
course.head(2)

,id,places,theme,cluster,score,place_title
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322,"[수목원테마공원(야시장길&LED공원), 카멜리아힐, 성읍민속마을, 아쿠아플라넷 제주]"
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116,"[성읍민속마을, 용두암, 카멜리아힐, 애월한담해안산책로, 수목원테마공원(야시장길&L..."


##샘플 100개로 연관규칙 확인

In [ ]:
sample = course.sample(100)['place_title']

In [ ]:
sample.values[0]

['우도(해양도립공원)',
 '성산일출봉(UNESCO 세계자연유산)',
 '천지연폭포',
 '섭지코지',
 '월정리해변',
 '오설록티뮤지엄',
 '김녕해수욕장',
 '애월해안도로',
 '위미리 동백나무군락']

In [ ]:
test_array = te.fit(sample.values).transform(sample.values)

In [ ]:
df = pd.DataFrame(test_array,columns = te.columns_)
df

,1100고지(휴게소),1100고지습지,1112도로,5.16도로숲터널,가파도,가파도 마라도 정기여객선,감귤박물관,거문오름(UNESCO 세계자연유산),건강과성박물관,검멀레해변,고근산(고공산),고산일과해안도로,공천포,곶자왈,곽지해수욕장,광치기해변,국립제주박물관,그랑블루 요트투어,금능해수욕장,김경숙해바라기농장,김녕 청굴물,김녕해수욕장,김녕해수욕장야영장,김녕해안도로,김영갑갤러리 두모악,낙천리아홉굿의자마을,남원큰엉해변,넥슨컴퓨터박물관,노루생태관찰원,다랑쉬오름(월랑봉),대평포구,더마파크,도두봉,도두항,도순다원,돈내코유원지,돔베낭골,라온승마클럽,렛츠런파크,마라도(마라해양도립공원),...,제주추사관,제주항공우주박물관,제주허브동산,제지기오름,조랑말체험공원,종달리수국길,종달리해안도로,주상절리대(중문대포해안),중문관광단지,중문색달해변(해산물),중문색달해수욕장,지미봉,차귀도유람선,천제연폭포,천지연폭포,청수곶자왈,취다선리조트,카멜리아힐,타보카 월정리투명카약,탑동해변공연장,테디베어뮤지엄제주,판포포구,표선세화해안도로,하고수동해변,하논마르,한라산영실코스,한라생태숲,한라수목원,한림공원,한림항,항파두리항몽유적지,해녀박물관,헬로키티아일랜드,협재굴(한림공원),협재해수욕장,화순금모래해수욕장,환상숲 곶자왈공원,황우지해안,휴애리 자연생활공원,흑돼지거리
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
96,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [ ]:
frequent_itemsets = apriori(df,min_support=0.01,use_colnames=True)
frequent_itemsets

In [ ]:
dataset = list(course['place_title'].values)

In [ ]:
dataset

In [ ]:
test_array = te.fit(dataset).transform(dataset)

In [ ]:
df = pd.DataFrame(test_array,columns=te.columns_)
df.head(3)

,1100고지(휴게소),1100고지습지,1112도로,5.16도로숲터널,가마오름,가뫼물,가파도,가파도 마라도 정기여객선,가파초등학교 마라분교장,각시바우오름,...,화순곶자왈생태탐방숲길,화순금모래해수욕장,화조원,환상숲 곶자왈공원,황금굴(한림공원),황우지해안,황우지해안 열두굴,후포해변,휴애리 자연생활공원,흑돼지거리
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
frequent_itemsets = apriori(df,min_support=0.01,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.020671,(1100고지(휴게소))
1,0.018523,(거문오름(UNESCO 세계자연유산))
2,0.014765,(건강과성박물관)
3,0.080268,(검멀레해변)
4,0.012886,(곶자왈)
...,...,...
3844,0.013960,"(우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 월정리해변, 카멜리아..."
3845,0.010201,"(우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 카멜리아힐, 천지연폭..."
3846,0.011544,"(오설록티뮤지엄, 성산일출봉(UNESCO 세계자연유산), 천지연폭포, 애월해안도로,..."
3847,0.010201,"(오설록티뮤지엄, 성산일출봉(UNESCO 세계자연유산), 월정리해변, 카멜리아힐, ..."


In [ ]:
check = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5) 

In [ ]:
check

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(검멀레해변),(성산일출봉(UNESCO 세계자연유산)),0.080268,0.343893,0.042148,0.525084,1.526882,0.014544,1.381522
1,(검멀레해변),(우도(해양도립공원)),0.080268,0.306309,0.055570,0.692308,2.260163,0.030984,2.254497
2,(우도등대),(검멀레해변),0.031946,0.080268,0.016913,0.529412,6.595514,0.014348,1.954430
3,(지미봉),(검멀레해변),0.034362,0.080268,0.017181,0.500000,6.229097,0.014423,1.839463
4,(하고수동해변),(검멀레해변),0.054497,0.080268,0.040805,0.748768,9.328303,0.036431,3.660892
...,...,...,...,...,...,...,...,...,...
5475,"(우도(해양도립공원), 오설록티뮤지엄, 월정리해변, 성산일출봉(UNESCO 세계자연...",(카멜리아힐),0.019866,0.288591,0.011812,0.594595,2.060339,0.006079,1.754810
5476,"(우도(해양도립공원), 오설록티뮤지엄, 월정리해변, 카멜리아힐, 협재해수욕장)",(성산일출봉(UNESCO 세계자연유산)),0.020671,0.343893,0.011812,0.571429,1.661648,0.004703,1.530917
5477,"(우도(해양도립공원), 오설록티뮤지엄, 성산일출봉(UNESCO 세계자연유산), 카멜...",(월정리해변),0.019597,0.283490,0.011812,0.602740,2.126142,0.006256,1.803629
5478,"(우도(해양도립공원), 월정리해변, 성산일출봉(UNESCO 세계자연유산), 카멜리아...",(오설록티뮤지엄),0.020671,0.324564,0.011812,0.571429,1.760605,0.005103,1.576018


In [ ]:
check.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000
mean,0.028012,0.310086,0.017216,0.635078,2.216240,0.008414,2.090754
std,0.020524,0.080007,0.011474,0.100545,0.848688,0.004777,0.884427
min,0.011007,0.040268,0.010201,0.500000,1.178052,0.001770,1.151141
25%,0.018255,0.263624,0.011544,0.549020,1.732274,0.005726,1.569285
50%,0.022282,0.306309,0.013691,0.616316,2.018975,0.007322,1.828441
75%,0.030336,0.343893,0.018255,0.706897,2.464552,0.009556,2.296655
max,0.343893,0.424430,0.188725,0.959184,13.226449,0.086249,16.074631


In [ ]:
high_lift = check[check['leverage']>0.01]

In [ ]:
high_lift.shape

(1197, 9)

In [ ]:
high_confidence = high_lift[high_lift['confidence']>0.7]

In [ ]:
high_confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(하고수동해변),(검멀레해변),0.054497,0.080268,0.040805,0.748768,9.328303,0.036431,3.660892
45,(이니스프리제주하우스),(오설록티뮤지엄),0.060671,0.324564,0.050201,0.827434,2.549372,0.030510,3.914066
85,"(하고수동해변, 섭지코지)",(검멀레해변),0.018255,0.080268,0.014228,0.779412,9.710063,0.012763,4.169450
90,"(성산일출봉(UNESCO 세계자연유산), 검멀레해변)",(우도(해양도립공원)),0.042148,0.306309,0.029799,0.707006,2.308150,0.016888,2.367598
94,"(하고수동해변, 성산일출봉(UNESCO 세계자연유산))",(검멀레해변),0.028725,0.080268,0.020134,0.700935,8.732379,0.017829,3.075352
...,...,...,...,...,...,...,...,...,...
4354,"(성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 카멜리아힐, 천지연폭포)",(섭지코지),0.022013,0.297987,0.017181,0.780488,2.619205,0.010622,3.198061
4398,"(섭지코지, 애월해안도로, 협재해수욕장, 천지연폭포)",(쇠소깍),0.025235,0.263624,0.017718,0.702128,2.663366,0.011066,2.472119
4400,"(협재해수욕장, 애월해안도로, 쇠소깍, 천지연폭포)",(섭지코지),0.023624,0.297987,0.017718,0.750000,2.516892,0.010678,2.808054
4492,"(우도(해양도립공원), 협재해수욕장, 쇠소깍, 천지연폭포)",(섭지코지),0.026309,0.297987,0.019597,0.744898,2.499770,0.011758,2.751893


In [ ]:
high_confidence['antecedents'] = high_confidence['antecedents'].apply(lambda x: list(x))
high_confidence['consequents'] = high_confidence['consequents'].apply(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
high_confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,[하고수동해변],[검멀레해변],0.054497,0.080268,0.040805,0.748768,9.328303,0.036431,3.660892
45,[이니스프리제주하우스],[오설록티뮤지엄],0.060671,0.324564,0.050201,0.827434,2.549372,0.030510,3.914066
85,"[하고수동해변, 섭지코지]",[검멀레해변],0.018255,0.080268,0.014228,0.779412,9.710063,0.012763,4.169450
90,"[성산일출봉(UNESCO 세계자연유산), 검멀레해변]",[우도(해양도립공원)],0.042148,0.306309,0.029799,0.707006,2.308150,0.016888,2.367598
94,"[하고수동해변, 성산일출봉(UNESCO 세계자연유산)]",[검멀레해변],0.028725,0.080268,0.020134,0.700935,8.732379,0.017829,3.075352
...,...,...,...,...,...,...,...,...,...
4354,"[성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 카멜리아힐, 천지연폭포]",[섭지코지],0.022013,0.297987,0.017181,0.780488,2.619205,0.010622,3.198061
4398,"[섭지코지, 애월해안도로, 협재해수욕장, 천지연폭포]",[쇠소깍],0.025235,0.263624,0.017718,0.702128,2.663366,0.011066,2.472119
4400,"[협재해수욕장, 애월해안도로, 쇠소깍, 천지연폭포]",[섭지코지],0.023624,0.297987,0.017718,0.750000,2.516892,0.010678,2.808054
4492,"[우도(해양도립공원), 협재해수욕장, 쇠소깍, 천지연폭포]",[섭지코지],0.026309,0.297987,0.019597,0.744898,2.499770,0.011758,2.751893


In [ ]:
high_confidence['len'] = high_confidence['antecedents'].apply(lambda x: len(x))
high_confidence.sort_values(by='len',ascending=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
high_confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,len
4,[하고수동해변],[검멀레해변],0.054497,0.080268,0.040805,0.748768,9.328303,0.036431,3.660892,1
45,[이니스프리제주하우스],[오설록티뮤지엄],0.060671,0.324564,0.050201,0.827434,2.549372,0.030510,3.914066,1
512,"[천제연폭포, 섭지코지]",[성산일출봉(UNESCO 세계자연유산)],0.028456,0.343893,0.022819,0.801887,2.331794,0.013033,3.311780,2
514,"[섭지코지, 천지연폭포]",[성산일출봉(UNESCO 세계자연유산)],0.101208,0.343893,0.075973,0.750663,2.182842,0.041168,2.631409,2
518,"[한림공원, 섭지코지]",[성산일출봉(UNESCO 세계자연유산)],0.030336,0.343893,0.023624,0.778761,2.264547,0.013192,2.965605,2
...,...,...,...,...,...,...,...,...,...,...
3878,"[월정리해변, 우도(해양도립공원), 섭지코지, 쇠소깍]",[성산일출봉(UNESCO 세계자연유산)],0.024161,0.343893,0.019866,0.822222,2.390927,0.011557,3.690604,4
3877,"[월정리해변, 우도(해양도립공원), 쇠소깍, 성산일출봉(UNESCO 세계자연유산)]",[섭지코지],0.027651,0.297987,0.019866,0.718447,2.411003,0.011626,2.493358,4
3814,"[섭지코지, 쇠소깍, 협재해수욕장, 오설록티뮤지엄]",[성산일출봉(UNESCO 세계자연유산)],0.036242,0.343893,0.027114,0.748148,2.175528,0.014651,2.605132,4
3966,"[성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 쇠소깍, 천지연폭포]",[섭지코지],0.034362,0.297987,0.026577,0.773438,2.595545,0.016338,3.098542,4


In [ ]:
high_confidence.loc[high_confidence['len']<3, ['antecedents','consequents']]

,antecedents,consequents
4,[하고수동해변],[검멀레해변]
45,[이니스프리제주하우스],[오설록티뮤지엄]
512,"[천제연폭포, 섭지코지]",[성산일출봉(UNESCO 세계자연유산)]
514,"[섭지코지, 천지연폭포]",[성산일출봉(UNESCO 세계자연유산)]
518,"[한림공원, 섭지코지]",[성산일출봉(UNESCO 세계자연유산)]
581,"[섭지코지, 애월해안도로]",[협재해수욕장]
588,"[섭지코지, 이니스프리제주하우스]",[오설록티뮤지엄]
649,"[이호테우해수욕장, 섭지코지]",[협재해수욕장]
735,"[성산일출봉(UNESCO 세계자연유산), 이니스프리제주하우스]",[오설록티뮤지엄]
762,"[우도(해양도립공원), 용두암]",[성산일출봉(UNESCO 세계자연유산)]


##10.10 - 다시 필터링해서 해보기

In [ ]:
process1_support = check[check['lift']>2]

In [ ]:
process1_support.shape

(2818, 9)

In [ ]:
process1_support['len'] = process1_support['antecedents'].apply(lambda x: len(x))
process1_support.sort_values(by='len',ascending=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
process1_support.loc[process1_support['len']<=3, ['antecedents','consequents']]

,antecedents,consequents
1,(검멀레해변),(우도(해양도립공원))
1035,(이니스프리제주하우스),"(협재해수욕장, 오설록티뮤지엄)"
63,(이중섭미술관),(이중섭거리)
58,(월정리해안도로),(월정리해변)
56,(하고수동해변),(우도(해양도립공원))
...,...,...
1898,"(성산일출봉(UNESCO 세계자연유산), 섭지코지, 이니스프리제주하우스)",(오설록티뮤지엄)
1897,"(오설록티뮤지엄, 월정리해변, 성산일출봉(UNESCO 세계자연유산))",(섭지코지)
2045,"(정방폭포, 성산일출봉(UNESCO 세계자연유산), 주상절리대(중문대포해안))",(섭지코지)
2046,"(정방폭포, 섭지코지, 주상절리대(중문대포해안))",(성산일출봉(UNESCO 세계자연유산))


###title -> id로 변환

In [ ]:
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...


In [ ]:
def inner_parse2(x):
  id = attraction[attraction['title']==x]['id'].values[0]
  return id

print(list(map(inner_parse2,['정방폭포', '섭지코지', '주상절리대(중문대포해안)'])))    

[17, 9, 16]


#특정 코스와 가장 연관성이 높은 품목 찾기

In [ ]:
input = ['오설록티뮤지엄', '월정리해변', '성산일출봉(UNESCO 세계자연유산)']
for val,attraction in zip(process1_support['antecedents'].values,process1_support['consequents'].values):
  match_rate = len(set(input).intersection(set(val))) /len(val)
  if match_rate >=1.0 :
    print('교집합',set(input).intersection(set(val)))
    print(val, "\t\t",attraction)

교집합 {'성산일출봉(UNESCO 세계자연유산)', '오설록티뮤지엄'}
frozenset({'성산일출봉(UNESCO 세계자연유산)', '오설록티뮤지엄'}) 		 frozenset({'섭지코지'})
교집합 {'월정리해변', '성산일출봉(UNESCO 세계자연유산)'}
frozenset({'월정리해변', '성산일출봉(UNESCO 세계자연유산)'}) 		 frozenset({'섭지코지'})
교집합 {'월정리해변', '성산일출봉(UNESCO 세계자연유산)', '오설록티뮤지엄'}
frozenset({'오설록티뮤지엄', '월정리해변', '성산일출봉(UNESCO 세계자연유산)'}) 		 frozenset({'섭지코지'})


###실제 코스데이터에 연관규칙 적용해서 확인해보기


In [ ]:
check.shape

(5480, 9)

In [ ]:
check.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000,5480.000000
mean,0.028012,0.310086,0.017216,0.635078,2.216240,0.008414,2.090754
std,0.020524,0.080007,0.011474,0.100545,0.848688,0.004777,0.884427
min,0.011007,0.040268,0.010201,0.500000,1.178052,0.001770,1.151141
25%,0.018255,0.263624,0.011544,0.549020,1.732274,0.005726,1.569285
50%,0.022282,0.306309,0.013691,0.616316,2.018975,0.007322,1.828441
75%,0.030336,0.343893,0.018255,0.706897,2.464552,0.009556,2.296655
max,0.343893,0.424430,0.188725,0.959184,13.226449,0.086249,16.074631


In [ ]:
check = check[check['lift']>1.5]

In [ ]:
def inner_parse(x):
  title = attraction[attraction['id']==x]['title'].values[0]
  return title

print(list(map(inner_parse,[1,2,3,4,5,6,7,8,9,10,11,12])))    

['사려니숲길', '우도(해양도립공원)', '성산일출봉(UNESCO 세계자연유산)', '비자림', '천지연폭포', '만장굴', '새별오름', '협재해수욕장', '섭지코지', '카멜리아힐', '거문오름(UNESCO 세계자연유산)', '함덕해수욕장']


In [ ]:
course['place_title'].values[0]

['수목원테마공원(야시장길&LED공원)', '카멜리아힐', '성읍민속마을', '아쿠아플라넷 제주']

In [ ]:

#input = ['사려니숲길', '우도(해양도립공원)', '성산일출봉(UNESCO 세계자연유산)', '비자림', '천지연폭포', '만장굴', '새별오름', '협재해수욕장', '섭지코지', '카멜리아힐', '거문오름(UNESCO 세계자연유산)', '함덕해수욕장']
entire_basket = []
from tqdm import tqdm
for input in tqdm(course['place_title'].values):
  same_basket = []
  for val,attraction in zip(check['antecedents'].values,check['consequents'].values):
    match_rate = len(set(input).intersection(set(val))) /len(val)
    if match_rate >=1.0 :
      # print('교집합',set(input).intersection(set(val)))
      #print(list(val), "\t\t",list(attraction))
      same_basket += list(attraction)
  same_basket = set(list(same_basket))
  recommend = list(same_basket - set(input))
  entire_basket.append(recommend)

100%|██████████| 3725/3725 [00:21<00:00, 177.12it/s]


In [ ]:
course['recommend'] = entire_basket

In [ ]:
course.head(10)

,id,places,theme,cluster,score,place_title,recommend
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322,"[수목원테마공원(야시장길&LED공원), 카멜리아힐, 성읍민속마을, 아쿠아플라넷 제주]",[]
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116,"[성읍민속마을, 용두암, 카멜리아힐, 애월한담해안산책로, 수목원테마공원(야시장길&L...","[성산일출봉(UNESCO 세계자연유산), 오설록티뮤지엄]"
2,3,"[58, 10, 19, 602, 29]","[0.2, 0.2, 0.6, 0.0, 0.2, 0.0, 0.2, 0.6, 0.8, ...",6,178099,"[성읍민속마을, 카멜리아힐, 애월한담해안산책로, 수목원테마공원(야시장길&LED공원)...",[]
3,141,"[1, 7, 8, 48, 22, 215]","[0.16666666666666666, 0.0, 0.5, 0.333333333333...",5,424938,"[사려니숲길, 새별오름, 협재해수욕장, 백약이오름, 오설록티뮤지엄, 성읍랜드]",[월정리해변]
4,156,"[1, 8, 174, 60, 184, 28, 223]","[0.0, 0.0, 0.5714285714285714, 0.4285714285714...",2,302233,"[사려니숲길, 협재해수욕장, 성이시돌목장, 산방산, 엉또폭포, 한림공원, 이호테우해...","[월정리해변, 카멜리아힐, 오설록티뮤지엄]"
5,216,"[2, 67, 3, 5, 4, 9, 60, 15, 16, 49, 270, 22, 28]","[0.0, 0.0, 0.3076923076923077, 0.6923076923076...",9,420002,"[우도(해양도립공원), 환상숲 곶자왈공원, 성산일출봉(UNESCO 세계자연유산), ...",[협재해수욕장]
6,4,"[202, 58, 37]","[0.0, 0.3333333333333333, 0.3333333333333333, ...",11,96780,"[제주승마공원, 성읍민속마을, 용두암]",[]
7,343,"[112, 9, 18, 20]","[0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 1.0, 0.75,...",5,230029,"[여미지식물원, 섭지코지, 에코랜드 테마파크, 천제연폭포]","[성산일출봉(UNESCO 세계자연유산), 쇠소깍]"
8,3227,"[2, 165, 8, 10, 76, 16, 49, 310, 23]","[0.0, 0.0, 0.4444444444444444, 0.5555555555555...",2,308804,"[우도(해양도립공원), 신창풍차해안도로, 협재해수욕장, 카멜리아힐, 새연교, 주상절...","[검멀레해변, 오설록티뮤지엄, 월정리해변, 성산일출봉(UNESCO 세계자연유산), ..."
9,3228,"[32, 3, 38, 6, 8, 9, 270, 15, 80, 16, 20, 22, ...","[0.07142857142857142, 0.07142857142857142, 0.3...",11,294152,"[제주절물자연휴양림, 성산일출봉(UNESCO 세계자연유산), 산방산탄산온천, 만장굴...","[우도(해양도립공원), 월정리해변, 카멜리아힐, 애월해안도로, 천지연폭포]"


In [ ]:
course['recommend_len'] = course['recommend'].apply(lambda x : len(x))
course['recommend_len'].describe()

count    3725.000000
mean        3.362148
std         1.836968
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max         9.000000
Name: recommend_len, dtype: float64

In [ ]:
for i in entire_basket:
  print(i)

[]
['성산일출봉(UNESCO 세계자연유산)', '오설록티뮤지엄']
[]
['월정리해변']
['월정리해변', '카멜리아힐', '오설록티뮤지엄']
['협재해수욕장']
[]
['성산일출봉(UNESCO 세계자연유산)', '쇠소깍']
['검멀레해변', '오설록티뮤지엄', '월정리해변', '성산일출봉(UNESCO 세계자연유산)', '천지연폭포', '섭지코지', '쇠소깍']
['우도(해양도립공원)', '월정리해변', '카멜리아힐', '애월해안도로', '천지연폭포']
['성산일출봉(UNESCO 세계자연유산)']
['성산일출봉(UNESCO 세계자연유산)', '천지연폭포']
['협재해수욕장', '쇠소깍']
['성산일출봉(UNESCO 세계자연유산)', '카멜리아힐']
[]
['월정리해변', '섭지코지', '애월해안도로', '천지연폭포']
['오설록티뮤지엄', '성산일출봉(UNESCO 세계자연유산)', '카멜리아힐', '섭지코지', '쇠소깍']
['하고수동해변', '성산일출봉(UNESCO 세계자연유산)', '월정리해변', '천지연폭포', '협재해수욕장']
['오설록티뮤지엄', '월정리해변', '협재해수욕장', '성산일출봉(UNESCO 세계자연유산)']
[]
['협재해수욕장']
['협재해수욕장', '쇠소깍', '오설록티뮤지엄']
['월정리해변', '성산일출봉(UNESCO 세계자연유산)', '섭지코지', '카멜리아힐']
['월정리해변', '섭지코지', '검멀레해변', '성산일출봉(UNESCO 세계자연유산)']
['섭지코지', '천지연폭포']
['월정리해변', '성산일출봉(UNESCO 세계자연유산)', '섭지코지', '오설록티뮤지엄']
['오설록티뮤지엄']
['이중섭거리', '협재해수욕장', '성산일출봉(UNESCO 세계자연유산)', '오설록티뮤지엄']
['성산일출봉(UNESCO 세계자연유산)', '쇠소깍']
['월정리해변', '쇠소깍', '성산일출봉(UNESCO 세계자연유산)']
['성산일출봉(UNESCO 세계자연유산)', '카멜리아힐', '섭지코지', '협재해수욕장', '쇠소깍'

##연관규칙 확장

In [ ]:
check2 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3) 
check2.shape

(12718, 9)

In [ ]:
check2.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,12718.000000,12718.000000,12718.000000,12718.000000,12718.000000,12718.000000,12718.000000
mean,0.038536,0.244911,0.017603,0.496362,2.316569,0.008384,1.656404
std,0.030300,0.100760,0.012386,0.144249,0.993577,0.004872,0.701119
min,0.011007,0.024698,0.010201,0.300000,0.713468,-0.009379,0.825566
25%,0.023087,0.149530,0.011544,0.375000,1.640345,0.005625,1.298484
50%,0.030872,0.263624,0.013691,0.469880,2.092367,0.007524,1.435458
75%,0.042685,0.324564,0.018792,0.593220,2.726090,0.009785,1.748485
max,0.424430,0.424430,0.188725,0.959184,13.226449,0.086249,16.074631


In [ ]:
check2[check2['lift']>1.5].shape

(10629, 9)

In [ ]:
check2 = check2[check2['lift']>1.5]

In [ ]:
entire_basket2 = []
from tqdm import tqdm
for input in tqdm(course['place_title'].values):
  same_basket = []
  for val,attraction in zip(check2['antecedents'].values,check2['consequents'].values):
    match_rate = len(set(input).intersection(set(val))) /len(val)
    if match_rate >=1.0 :
      # print('교집합',set(input).intersection(set(val)))
      #print(list(val), "\t\t",list(attraction))
      same_basket += list(attraction)
  same_basket = set(list(same_basket))
  recommend = list(same_basket - set(input)) #이미 일정에 있는애들은 연관규칙 기반 추천에서 제외
  entire_basket2.append(recommend)

100%|██████████| 3725/3725 [00:38<00:00, 95.93it/s]


In [ ]:
course['more_recommend2'] = entire_basket2
course.sample(10)

,id,places,theme,cluster,score,place_title,recommend,recommend_len,more_recommend2,recommend_len2
619,615,"[161, 165, 105, 554, 175, 532, 1081, 186, 59, ...","[0.2, 0.0, 0.3, 0.5, 0.1, 0.0, 0.0, 0.9, 0.4, ...",3,39506,"[한라산어리목코스, 신창풍차해안도로, 1100고지(휴게소), 김녕해안도로, 윗세오름...",[협재해수욕장],1,"[성산일출봉(UNESCO 세계자연유산), 섭지코지, 쇠소깍, 협재해수욕장]",1
608,604,"[10, 75, 108, 7]","[0.5, 0.0, 0.25, 0.25, 0.5, 0.0, 0.5, 0.0, 1.0...",4,252587,"[카멜리아힐, 방주교회, 본태박물관, 새별오름]",[],0,[성이시돌목장],0
749,742,"[130, 3, 2, 5, 6, 8, 9, 10, 13, 15, 16, 21, 22...","[0.0, 0.0, 0.42857142857142855, 0.571428571428...",11,335517,"[제주올레 7코스, 성산일출봉(UNESCO 세계자연유산), 우도(해양도립공원), 천...",[애월해안도로],1,"[검멀레해변, 애월해안도로, 사려니숲길, 정방폭포, 김녕해수욕장, 천제연폭포, 비자림]",1
1736,1717,"[40, 32, 275, 109]","[0.25, 0.0, 0.75, 0.0, 0.25, 0.0, 0.0, 0.75, 1...",10,96629,"[보롬왓, 제주절물자연휴양림, 제주추사관, 렛츠런팜제주]",[],0,[],0
1634,1612,"[1, 2, 99, 3, 37, 8, 27, 175, 16, 49, 15, 20, ...","[0.0, 0.07142857142857142, 0.2857142857142857,...",2,385736,"[사려니숲길, 우도(해양도립공원), 제주빅볼랜드, 성산일출봉(UNESCO 세계자연유...","[오설록티뮤지엄, 월정리해변, 카멜리아힐, 천지연폭포, 섭지코지]",5,"[용머리해안, 오설록티뮤지엄, 월정리해변, 카멜리아힐, 천지연폭포, 섭지코지, 만장...",5
3241,33,"[58, 9, 10, 19, 602, 29]","[0.16666666666666666, 0.16666666666666666, 0.5...",5,215726,"[성읍민속마을, 섭지코지, 카멜리아힐, 애월한담해안산책로, 수목원테마공원(야시장길&...","[오설록티뮤지엄, 월정리해변, 협재해수욕장, 성산일출봉(UNESCO 세계자연유산)]",4,"[우도(해양도립공원), 에코랜드 테마파크, 오설록티뮤지엄, 월정리해변, 성산일출봉(...",4
2638,2614,"[3, 5, 134, 10, 78, 18, 22, 310]","[0.125, 0.0, 0.375, 0.5, 0.125, 0.125, 0.125, ...",5,332540,"[성산일출봉(UNESCO 세계자연유산), 천지연폭포, 검멀레해변, 카멜리아힐, 제주...","[우도(해양도립공원), 월정리해변, 섭지코지, 협재해수욕장, 쇠소깍]",5,"[우도(해양도립공원), 용머리해안, 월정리해변, 성읍민속마을, 애월해안도로, 주상절...",5
438,435,"[1, 223, 3, 8, 9, 10, 75, 203, 141, 270, 46, 2...","[0.0625, 0.0625, 0.3125, 0.5625, 0.0625, 0.0, ...",7,283098,"[사려니숲길, 이호테우해수욕장, 성산일출봉(UNESCO 세계자연유산), 협재해수욕장...","[우도(해양도립공원), 검멀레해변, 오설록티뮤지엄, 월정리해변, 천지연폭포, 쇠소깍]",6,"[우도(해양도립공원), 검멀레해변, 오설록티뮤지엄, 월정리해변, 천지연폭포, 애월해...",6
2367,2345,"[2, 3, 5, 134, 8, 9, 10, 13, 17, 18, 20, 25, 1...","[0.1, 0.0, 0.35, 0.55, 0.05, 0.0, 0.05, 0.9, 0...",7,288471,"[우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 천지연폭포, 검멀레해...","[애월해안도로, 쇠소깍, 오설록티뮤지엄]",3,"[용머리해안, 오설록티뮤지엄, 애월해안도로, 곽지해수욕장, 주상절리대(중문대포해안)...",3
1931,1912,"[98, 4, 8, 202, 13, 174, 434, 22, 25, 31]","[0.1, 0.0, 0.8, 0.1, 0.0, 0.1, 0.0, 0.9, 0.4, ...",1,240677,"[월령선인장군락지, 비자림, 협재해수욕장, 제주승마공원, 월정리해변, 성이시돌목장,...","[우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 카멜리아힐, 섭지코지...",5,"[우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 카멜리아힐, 애월해안...",5


In [ ]:
course['recommend_len2'] = course['more_recommend2'].apply(lambda x: len(x))

In [ ]:
course['recommend_len2'].describe()

count    3725.000000
mean        8.128591
std         3.092358
min         0.000000
25%         7.000000
50%         9.000000
75%        10.000000
max        16.000000
Name: recommend_len2, dtype: float64

In [ ]:
check2.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.10_association_rule.csv")

In [ ]:
check2['antecedents'] = check2['antecedents'].apply(lambda x:list(x))
check2['consequents'] = check2['consequents'].apply(lambda x:list(x))

In [ ]:
check2.to_pickle("/content/drive/MyDrive/yeoreodigm/data_files/10.10_association_rule.pkl")

In [ ]:
check2['test'] = check2['antecedents'].apply(lambda x: list(map(inner_parse2,x)))

TypeError: ignored

In [ ]:
check

In [ ]:
check3 = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.10_association_rule.csv",index_col=0)

In [ ]:
for i in check3.sample(10)['antecedents'].values:
  print(i)

['정방폭포', '섭지코지']
['월정리해변', '성산일출봉(UNESCO 세계자연유산)', '섭지코지', '카멜리아힐']
['월정리해변', '성산일출봉(UNESCO 세계자연유산)', '카멜리아힐', '쇠소깍']
['성산일출봉(UNESCO 세계자연유산)', '협재해수욕장', '쇠소깍']
['애월해안도로', '카멜리아힐', '쇠소깍']
['섭지코지', '천지연폭포', '협재해수욕장', '오설록티뮤지엄']
['우도(해양도립공원)', '쇠소깍', '비자림']
['우도(해양도립공원)', '섭지코지', '쇠소깍', '협재해수욕장']
['월정리해변', '성산일출봉(UNESCO 세계자연유산)', '협재해수욕장', '이호테우해수욕장']
['이호테우해수욕장', '성산일출봉(UNESCO 세계자연유산)', '협재해수욕장']
